In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangGraph Session05 - Standalone Agent"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [13]:
# 🔒 Fully Working LangGraph Agent with Manual Tool Handling

import random, requests
import nest_asyncio
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated

nest_asyncio.apply()

# 🛠️ Define Tools
@tool
def get_weather(city: str) -> str:
    """Returns a dummy weather report for a given city."""
    return f"The weather in {city} is sunny."

@tool
def wiki_search(query: str) -> str:
    """Searches Wikipedia for a summary of the given query."""
    try:
        response = requests.get(f"https://en.wikipedia.org/api/rest_v1/page/summary/{query.replace(' ', '_')}")
        data = response.json()
        return data.get("extract", "No summary available.")
    except Exception as e:
        return f"Wiki search failed: {e}"

@tool
def fun_fact(topic: str) -> str:
    """Returns a fun fact about the given topic."""
    return f"Did you know that {topic} has a fascinating history?"

@tool
def random_color(colors: list[str]) -> str:
    """Randomly selects a color from a given list of strings."""
    print(f"[TOOL] Choosing from: {colors}")
    return random.choice(colors) if colors else "No colors provided."


# 🔧 Setup toolbelt and model
tool_belt = [get_weather, wiki_search, fun_fact, random_color]
tool_dict = {tool.name: tool for tool in tool_belt}

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)
model = model.bind_tools(tool_belt)

# 🧠 Define Agent State
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# 🧠 Agent Node
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

# 🛠️ Tool Execution Node
def tool_executor(state):
    messages = state['messages']
    last = messages[-1]
    tool_messages = []

    for call in last.tool_calls:
        tool_fn = tool_dict.get(call["name"])
        if tool_fn:
            result = tool_fn.invoke(call["args"])
            tool_messages.append(ToolMessage(
                tool_call_id=call["id"],
                content=result
            ))
    # return {"messages": messages}

    return {"messages": messages + tool_messages}

# 🧩 Build LangGraph
graph = StateGraph(AgentState)
graph.add_node("agent", call_model)
graph.add_node("action", tool_executor)

def decide_next(state):
    return "action" if state["messages"][-1].tool_calls else END

graph.add_conditional_edges("agent", decide_next, {"action": "action", END: END})
graph.set_entry_point("agent")

compiled_graph = graph.compile()

# 🧪 Prompt Examples
prompts = [
    "What is the weather in Parsippany?",
    "Search Wikipedia for information about Declarationism.",
    "Tell me a fun fact about ice cream.",
    "Pick a random color from ['red-blue-yellow', 'yellow-blue-red', 'blue-red-yellow']."
]


# 🧪 Streaming Execution
import asyncio

async def run_streaming():
    for prompt in prompts:
        print(f"\n🟢 Prompt: {prompt}")
        inputs = {"messages": [HumanMessage(content=prompt)]}
        async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
            for node, values in chunk.items():
                print(f"📍 Node: {node}")
                pprint(values["messages"])
        print(values["messages"][-1].content)
        print("\n\n")

await run_streaming()



🟢 Prompt: What is the weather in Parsippany?
📍 Node: agent
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_U9ma8SITcphTJBsPzifVL7TN', 'function': {'arguments': '{"city":"Parsippany"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 130, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BsHpyVLIstjrixetz6WOG2PkloIa0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--db2d0111-939b-4d45-a7e6-cb1a822319e1-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Parsippany'}, 'id': 'call_U9ma8SITcphTJBsPzifVL7TN', 'type': 'tool_call'}], usage_metadata={'input_tokens': 130, 'output_